# Homework 5 - Visit the Wikipedia hyperlinks graph!

## Group #2 Riccardo Cervelli, Davide Facchinelli

![WIKI](https://www.balcanicaucaso.org/var/obc/storage/images/articoli-da-pubblicare-2/turchia-oscurata-wikipedia/1221571-1-ita-IT/Turchia-oscurata-Wikipedia.jpg)

In [1]:
from collections import defaultdict
import statistics
import pickle
from tqdm import tqdm
import numpy as np
import igraph

The Goal of this assignment is to perform an analysis of the Wikipedia Hyperlink graph. In particular, given extra information about the categories to which an article belongs to, we ranked the articles according to some criteria that we'll explain you later.
For this purpose we use the Wikipedia graph released by the SNAP group and you can check the files that we used [here](https://drive.google.com/file/d/1ghPJ4g6XMCUDFQ2JPqAVveLyytG8gBfL/view?usp=sharing), in order to compute find:
- The number of nodes.
- The number of edges.
- The average node degree.

We are building our graph reading the file.

In [2]:
# we inizialize our graph as a default dictionary
graph = defaultdict(list)

# and populate it from our file
with open('wiki-topcats-reduced.txt') as f:
    for row in f.readlines():
        edges = list(map(int,row.split('\t')))
        graph[edges[0]].append(edges[1])

We are building the set of all the vertices.

In [3]:
total_vertex = list(graph.keys())
for k in graph.keys():
    total_vertex+=graph[k]
total_vertex = set(total_vertex)
indicizer = sorted(list(total_vertex))

# we prepare a dictonary that given a vertex it gives us it's index
vertex_to_index = {indicizer[i]:i for i in range(len(indicizer))}

We want to calculate the Number of vertex :

In [4]:
n_vert = len(total_vertex)

We want to calculate  Number of edges :

In [5]:
n_edges = sum(len(graph[k]) for k in total_vertex)

We want to calculate the Density of our graph ($D$) :
$D ={\frac  {n_v}{n_v-1}}$

In [6]:
density = n_edges/(n_vert*(n_vert-1))

And finally the Avreage node degree ($AND$) :  $AND ={\frac  {2n_e}{n_v}}$

In [7]:
avreage_degree = 2*n_edges/n_vert

#### Our Data

In [8]:
print('The Number of vertex is : ',n_vert)
print('The Number of edges is :',n_edges)
print('The density of our graph is :',density)
print('The Average node Degree is :',avreage_degree)

The Number of vertex is :  461193
The Number of edges is : 2645247
The density of our graph is : 1.2436602635647606e-05
The Average node Degree is : 11.471323285479182


#### First question : Is the graph dense?
In mathematics, a dense graph is a graph in which the number of edges is close to the maximal number of edges. We calculated the density as $D={\frac  {|E|}{|V|\,(|V|-1)}}$ and we can see that is quite low. So no, the graph is not dense. We will show you later our plots, where yourselfe can observe the density of our graph.

----------

Let's go to the next step.
- Load the categories (take into account all the categories that have a number of articles greater than 3500).

In [9]:
# we inizialize it as an empty dictonary
category_dic = {}

# and load the category from the file
with open('wiki-topcats-categories.txt') as f:
    for row in f.readlines():
        category = row[:-2].split(' ')
        elements = set(map(int,category[1:]))
        # checking if the category is big enough
        if len(elements)>3500:
            # and taking only verteces that exist in our starting graph
            category_dic[category[0][9:-1]] = elements & total_vertex

In [10]:
# we create a temporary second dictonary
category_dic2 = {}
# and we save in it only the category that are not empty
for k in category_dic.keys():
    if category_dic[k]:
        category_dic2[k] = category_dic[k]
category_dic = category_dic2
del category_dic2

In [11]:
print('We can use only',len(category_dic),'categories.')

We can use only 35 categories.


Let's take a look at how big these categories are.

In [12]:
for item in category_dic.items():
    print(item[0],len(item[1]))

English_footballers 7537
The_Football_League_players 7813
Association_football_forwards 5097
Association_football_goalkeepers 3736
Association_football_midfielders 5826
Association_football_defenders 4589
Living_people 348299
Year_of_birth_unknown 2536
Harvard_University_alumni 5548
Major_League_Baseball_pitchers 5193
Members_of_the_United_Kingdom_Parliament_for_English_constituencies 6491
Indian_films 5567
Year_of_death_missing 4122
English_cricketers 3275
Year_of_birth_missing_(living_people) 28498
Rivers_of_Romania 7728
Main_Belt_asteroids 11659
Asteroids_named_for_people 4895
English-language_albums 4760
English_television_actors 3361
British_films 4422
English-language_films 22462
American_films 15158
Fellows_of_the_Royal_Society 3446
People_from_New_York_City 4615
American_Jews 3411
American_television_actors 11531
American_film_actors 13865
Debut_albums 7561
Black-and-white_films 10758
Year_of_birth_missing 4346
Place_of_birth_missing_(living_people) 5532
Article_Feedback_Pilot 

Importing a function from our file we get the list of categories ordered.

Now we are going to define  a BFS function implemented from scratch ,finding all the shortest distance from a vertex to all the others.
Notice that if there is not a way to go from a node to another we assigned as distance the value $-1$. Classically in the literature the value assigned in those cases is $\infty$, and further in the code we will substitue the $-1$ value with that.

In [13]:
# calssic BFS algorithm

def path_len(start,graph,indicizer):
    
    verteces = defaultdict(lambda:(False,np.infty))
    verteces[start] = (True,0)
    
    queue = [start]

    while queue:
        actual = queue.pop(0)
        d = verteces[actual][1]
        for child in graph[actual]:
            if not verteces[child][0]:
                verteces[child] = (True,d+1)
                queue.append(child)
    result = []
    for k in indicizer:
        result.append(verteces[k][1])
    return result

Here we input our input category.

In [15]:
input_category = input('Input category: ')

Input category: Year_of_birth_unknown


We get all the distances from the vertex of the input category to all the other vertices. As the files are really big we save them on the drive because we tryed to don't overload the RAM.


In [15]:
# we apply the BFS algorithm to each element of the input_category
vdist = dict()
# we inzialize two counters
# one to count how many start vertex we have stored
vx = 0
# the other will set the name of the file where we are going to save them
files = 0
for v in tqdm(category_dic[input_category]):
    # we compure the distance from v to each other vertex
    vdist[v] = path_len(v,graph,indicizer)
    vx+=1
    # and each 100 vertex we save it on a file
    if vx == 100:
        files+=1
        with open('dist_dict_'+str(files)+'.pkl','wb') as f:
            pickle.dump(vdist,f, pickle.HIGHEST_PROTOCOL)
        # we reset our variable and counter
        vdist = dict()
        vx = 0

with open('dist_dict_'+str(files+1)+'.pkl','wb') as f:
            pickle.dump(vdist,f, pickle.HIGHEST_PROTOCOL)

100%|████████████████████████████████████████████████████████████████████████████| 2536/2536 [2:38:49<00:00,  1.51s/it]


We load the file and proceed to get the ordered category list.

In [18]:
l = []
files = int(len(category_dic[input_category])/100)+1
# we get the distance from the input category to each other
for category in tqdm(category_dic.keys()):
    infty_c = 0
    dist_list = []
    # we iterate on the file contaning the distances
    for i in range(1,files+1):
        with open('dist_dict_'+str(i)+'.pkl','rb') as f:
            vdist = pickle.load(f)
        for v in vdist.keys():
            for w in category_dic[category]:
                d = vdist[v][vertex_to_index[w]]
                if d == np.infty:
                    infty_c+=1
                else:
                    dist_list.append(d)
    dist_list = sorted(dist_list)
    s = len(dist_list)+infty_c
    try:
        if s%2 == 0:
            l.append((category,(dist_list[int(s/2)]+dist_list[int(s/2)-1])/2))
        else:
            l.append((category,dist_list[int((s-1)/2)]))
    except IndexError:
        l.append((category,np.infty))
    # and we store them in a list
    l.append((category,np.median(dist_list)))
# we save the ordered list
ordered_cat = sorted(l,key= lambda x:x[1])

# we move the input category to the first position and assign to him weight 0
ordered_cat.pop(list(map(lambda x: x[0],ordered_cat)).index(input_category))
ordered_cat.insert(0,(input_category,0))

# we save our newly found item
with open('olist_cat.pkl','wb') as f:
    pickle.dump(ordered_cat,f, pickle.HIGHEST_PROTOCOL)


100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [55:38<00:00, 78.28s/it]


As it can be seen from our code, we decided to consider the median of the values on each couple of point, even if they are not linked. In this way we still consider that two categories are far away if they have any element in common.

In [14]:
with open('olist_cat.pkl','rb') as f:
    ordered_cat = pickle.load(f)

for item in ordered_cat:
    print(item[0],item[1] if item[1] != np.infty else 'Infinity')

Year_of_birth_unknown 0
Harvard_University_alumni 6.0
English_television_actors 6.0
British_films 6.0
English-language_films 6.0
English-language_films 6.0
American_films 6.0
Fellows_of_the_Royal_Society 6.0
People_from_New_York_City 6.0
American_Jews 6.0
American_television_actors 6.0
American_film_actors 6.0
American_film_actors 6.0
Black-and-white_films 6.0
Article_Feedback_Pilot 6.0
American_military_personnel_of_World_War_II 6.0
English_footballers 7.0
The_Football_League_players 7.0
Association_football_forwards 7.0
Association_football_goalkeepers 7.0
Association_football_midfielders 7.0
Association_football_defenders 7.0
Living_people 7.0
Major_League_Baseball_pitchers 7.0
Members_of_the_United_Kingdom_Parliament_for_English_constituencies 7.0
Members_of_the_United_Kingdom_Parliament_for_English_constituencies 7.0
Indian_films 7.0
Indian_films 7.0
Year_of_death_missing 7.0
English_cricketers 7.0
Year_of_birth_missing_(living_people) 7.0
Rivers_of_Romania 7.0
English-language_al

We compute which vertex are in our blocks.

In [15]:
# we make a list of vertex to assign
unassigned_vertex = total_vertex.copy()

block_list = []
# we populate each category
for cat in ordered_cat:
    # we check if we still have verteces to assign
    if not unassigned_vertex:
        break
    # and we assign all the still free verteces to his category
    else:
        to_insert = unassigned_vertex & category_dic[cat[0]]
        block_list.append((cat[0],to_insert))
        # we delete from our set of free verteces the asigned ones
        unassigned_vertex-=to_insert
# we delete categories that did not get any vertex
tlist = []
for item in block_list:
    if item[1]:
        tlist.append(item)
block_list = tlist

We give a look to which blocks we have and how big they are.

In [16]:
for item in block_list:
    print(item[0],len(item[1]))

Year_of_birth_unknown 2536
Harvard_University_alumni 5544
English_television_actors 3359
British_films 4422
English-language_films 18909
American_films 4598
Fellows_of_the_Royal_Society 3425
People_from_New_York_City 4432
American_Jews 2909
American_television_actors 10495
American_film_actors 4587
Black-and-white_films 3620
Article_Feedback_Pilot 3398
American_military_personnel_of_World_War_II 3183
English_footballers 7514
The_Football_League_players 2790
Association_football_forwards 3366
Association_football_goalkeepers 2837
Association_football_midfielders 4009
Association_football_defenders 2896
Living_people 306928
Major_League_Baseball_pitchers 1931
Members_of_the_United_Kingdom_Parliament_for_English_constituencies 5024
Indian_films 5399
Year_of_death_missing 3544
English_cricketers 1820
Year_of_birth_missing_(living_people) 74
Rivers_of_Romania 7728
English-language_albums 4746
Debut_albums 6647
Year_of_birth_missing 2451
Place_of_birth_missing_(living_people) 29
Windows_game

We load the reversed graph, that is, to each key is associate the entering edge, not the exiting one.

In [17]:
entring_edge = defaultdict(list)

with open('wiki-topcats-reduced.txt') as f:
    for row in f.readlines():
        edges = list(map(int,row.split('\t')))
        entring_edge[edges[1]].append(edges[0])

We associate to each veretex a weight.

In [18]:
# we inizialize a dictionary to be populated with as keys our vertex, and as value their weight
w_dic = defaultdict(int)
# we have a set of all the vertex in the categories we are considering up to now
considering = set()
for i in range(len(block_list)):
    # we upload the considering set with all the new vertex
    considering|=block_list[i][1]
    # for each element in our block
    for v in block_list[i][1]:
        # and for each considered element that has an entering edge in our element
        for w in considering & set(entring_edge[v]):
                # we appload his value
                if w in block_list[i][1]:
                    w_dic[v]+=1
                else:
                    w_dic[v]+=w_dic[w]

We wrote an algorithm to show you the correlation between the first two blocks using igraph.plot. The size of the images are really big, and we can not not load them inside this notebook. We saved them in the svg format and loaded them in the github repository. As the svg format is naturally readible in a browser we placed the link to the visualization of the file.

We display in different colors different categories, and place inside each vertex his total weight.

In [48]:
# we set how many block we want to plot
n_block = 2
# the size of the plot
size = 10000

# we get a subset of our total list of vertex corresponding to this subgraph
to_plot = {}
our_vert = set()
for i in range(n_block):
    our_vert|=block_list[i][1]
for k in our_vert:
    to_plot[k] = set(graph[k]) & our_vert

In [49]:
# we create an empty graph
g = igraph.Graph(directed = True)
# we add the vertices
g.add_vertices(list(map(str,to_plot.keys())))
# the edges
g.add_edges([(str(s),str(g))  for s in to_plot.keys() for g in to_plot[s]])
# we place as label the weight of each vertix
g.vs['label'] = [w_dic[int(k)] for k in g.vs['name']]
# we prepare a list with as many color as blocks plotted
colorlist = ['red', 'blue']
# and we associate each color to the vertex of a block
vertex_color = []
for k in g.vs['name']:
    for i in range(n_block):
        if int(k) in block_list[i][1]:
            vertex_color.append(colorlist[i])
            break
g.vs['color'] = vertex_color

In [50]:
g.write_svg('plot1.svg',width= size, height= size, vertex_size=7)

[CLICK HERE TO CHECK OUR FIRST PLOT !](https://raw.githubusercontent.com/CervelliRic/ADM-HW5/master/plot1.svg?sanitize=true)

Careful, the image is very big, you will need to move the point of view because at first it will be focused in the top left corener, potentially empty.

In [51]:
# we prepare the layout
lay = g.layout_lgl()

In [52]:
g.write_svg('plot2.svg',width= size, height= size, vertex_size=7,layout=lay)

[AND HERE TO CHECK OUR SECOND PLOT !](https://raw.githubusercontent.com/CervelliRic/ADM-HW5/master/plot2.svg?sanitize=true)

Careful, the image is very big, you will need to move the point of view because at first it will be focused in the top left corener, potentially empty.

Is not very informative what we obtained because the graph is huge, but we can still see something.

We plotted two graphs. The first highlight how many unlinked vertex we have obtain.With the second let us see in detail the weight of the verteces in the middle of the first one. 

We can notice there are a lot of verteces with weight $0$ (it is also implied by the low density). We decided to plot only vertex with positive weight, more interesting for us and certainly we can have a better the idea of what we are observing.

In [37]:
to_plot2 = {}

for k in to_plot.keys():
    if w_dic[k] > 0:
        to_plot2[k] = [e for e in to_plot[k] if w_dic[e] > 0]

# we create an empty graph
g = igraph.Graph(directed = True)
# we add the vertices
g.add_vertices(list(map(str,to_plot2.keys())))
# the edges
g.add_edges([(str(s),str(g))  for s in to_plot2.keys() for g in to_plot2[s]])
# we place as label the weight of each vertix
g.vs['label'] = [w_dic[int(k)] for k in g.vs['name']]
# we prepare a list with as many color as blocks plotted
colorlist = ['red', 'blue']
# and we associate each color to the vertex of a block
vertex_color = []
for k in g.vs['name']:
    for i in range(n_block):
        if int(k) in block_list[i][1]:
            vertex_color.append(colorlist[i])
            break
g.vs['color'] = vertex_color

In [38]:
g.write_svg('redplot1.svg',width= size, height= size, vertex_size=7)

[HERE WE ARE PLOTTING ONLY THE VERTEX WITH POSITIVE WEIGHT.](https://raw.githubusercontent.com/CervelliRic/ADM-HW5/master/redplot1.svg?sanitize=true)

Careful, the image is very big, you will need to move the point of view because at first it will be focused in the top left corener, potentially empty.

In [39]:
# we prepare the layout
lay = g.layout_lgl()

g.write_svg('redplot2.svg',width= size, height= size, vertex_size=7, layout=lay)

[POSITIVE WEIGHT OF THE VERTECES IN THE MIDDLE.](https://raw.githubusercontent.com/CervelliRic/ADM-HW5/master/redplot2.svg?sanitize=true)

Careful, the image is very big, you will need to move the point of view because at first it will be focused in the top left corener, potentially empty.